In [ ]:
!pip install -q -U git+https://github.com/huggingface/transformers

In [ ]:
! pip install flash-attn --no-build-isolation

In [ ]:
import torch
from transformers import AutoTokenizer
from transformers import AutoModelForCausalLM


tokenizer=AutoTokenizer.from_pretrained('Mistral-7B-Instruct-v0.1')

model = AutoModelForCausalLM.from_pretrained(
    'Mistral-7B-Instruct-v0.1',
    torch_dtype = torch.bfloat16,
    device_map = "auto",
    trust_remote_code = True,
)

In [ ]:
%%time

def generate_essay(prompt):
    messages = [{
        "role":"user",
        "content": prompt
    }]

    model_inputs = tokenizer.apply_chat_template(messages, return_tensors = "pt").to('cuda')
    
    # Setting `pad_token_id` to `eos_token_id` for open-ended generation.
    with torch.no_grad():
        generated_ids = model.generate(
            model_inputs,
            max_new_tokens = 7500,
            do_sample = True,
            pad_token_id = tokenizer.eos_token_id
        )

    decoded = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)
    text = decoded[0].split("[/INST]")[1]
    return text

In [ ]:
prompt = '''You are a student working on the following assignment.

Create an essay based on the following topic in no more than 1000 words.

Topic: Car-Free cities.
'''

In [ ]:
text = generate_essay(prompt)
text